In [13]:
# load the sequence and probabilities into memory
sequence_filename = "chr22.maf.ancestors.42000000.complete.boreo.fa.txt"
probabilities_filename = "chr22.maf.ancestors.42000000.complete.boreo.conf.txt"

with open(sequence_filename) as f:
    sequence = f.readline()

with open(probabilities_filename) as f:
    probabilities = f.readline().split()

# make all probabilities float
n = len(probabilities)
for i in range(n):
    probabilities[i] = float(probabilities[i])

In [14]:
# set random seed
import random
import math
random.seed(1)

In [15]:
# generate test sequence

In [16]:
def get_test_sequence(length,prob_insertion,prob_deletion, prob_point):
    nucleotides = ['A', 'G', 'C', 'T']
    
    # get random starting position
    
    len_sequence = len(sequence)
    # len of probabilities should be the same
    
    start_index = random.randrange(len_sequence-length-1)
    
    query = list()
    
    for i in range(start_index, start_index+length):
        insertion = list()
        extra_nucleotide = ""
        
        if (probabilities[i] > random.random()):
                nucleotide = sequence[i]
        else:
            temp_nucleotides = ['A', 'G', 'C', 'T']
            temp_nucleotides.remove(sequence[i])
            nucleotide = temp_nucleotides[random.randrange(3)]
        
        insertion_deletion_prob = random.random()
        
        if insertion_deletion_prob < prob_insertion:
            # if we are within probability of getting an insertion 
            # length follows a geometric distribution
            insertion_length = int(math.log(random.random()/prob_insertion)/math.log(1-prob_insertion) + 1)
            for i in range(insertion_length):
                extra_nucleotide = nucleotides[random.randrange(4)] 
                insertion.append(extra_nucleotide)

        elif insertion_deletion_prob < prob_insertion + prob_deletion:
            # if we are within probability of getting a deletion
            nucleotide = ""
           
        elif insertion_deletion_prob < prob_insertion + prob_deletion + prob_point:
            nucleotide = nucleotides[random.randrange(4)]
  
        query.append(nucleotide + "".join(insertion))
    
    return ["".join(query),start_index]

In [17]:
get_test_sequence(100, 0.0005, 0.0005, 0.005)

['GCTAGCTAGAGTTTTACCTTTTAGGGAAGAGAACTTAGAAAATGGAGATTGGGTCTTTGACCAACAGCAAATAGAGGAAATTGATTTTCCATATATTTCT',
 140891]

In [18]:
# test_name = "test_sequence"
# new_filename = test_name + ".fa"

# length = 300
# prob_insertion = 0.01
# prob_deletion = 0.01

# test_sequence_list = get_test_sequence(length,prob_insertion,prob_deletion)

# with open(new_filename, 'w') as f:
#     f.write(">" + test_name + " " + str(test_sequence_list[1]) + " " + str(length) + "\n")
#     f.write(test_sequence_list[0])

In [19]:
import blast
import os
import time

In [20]:
def gen_test_probabilistic_blast(test_name: str,iterations: int):
    
    cur_dir = os.getcwd()
    os.mkdir(test_name)
    
    for i in range(iterations):
        
        new_filename = test_name +  str(i) + ".fa"
        new_path_to_file = os.path.join(cur_dir, test_name, new_filename)

        length = 400
        prob_insertion = 0.0005
        prob_deletion = 0.0005
        prob_point = 0.005

        test_sequence_list = get_test_sequence(length,prob_insertion,prob_deletion,prob_point)

        with open(new_path_to_file, 'w') as f:
            f.write(">" + test_name + " " + str(test_sequence_list[1]) + " " + str(length) + "\n")
            f.write(test_sequence_list[0])
        

In [19]:
results = gen_test_probabilistic_blast("test_testing_suite",100)

In [16]:
print(results)

[0, 45, 0, 100]


In [9]:
# test 1: 0 10 100
# test 2: 0 45 100
# test 3: 0 45 100

# test 3 with epsilon 0 : 0 0 100
# test 3 with epsilon 10: 0 46 100

# higher epsilon is better

# test 3 with top ten: 0 45 0 100
# test 4: 0 45 0 100
# test 5: 0 45 0 100

In [21]:
def test_probabilistic_blast(test_name: str,pblast: blast.PBlast):
    
    exact_index_matches = 0
    contains_match = 0
    contains_start = 0
    contains_end = 0
    total_time = 0
    topten = 0
    topten_contains_start = 0
    topten_contains_end = 0
    
    
    cur_dir = os.getcwd()
    test_dir = os.path.join(cur_dir, test_name) 
    
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(test_dir):
        for file in f:
            if '.fa' in file:
                files.append(os.path.join(r, file))
    
    
    for file in files:
        
        #read first line of fasta file
        with open(file, 'r') as f:
            info = f.readline().split()
            start_index = int(info[1])
            length = int(info[2])
        
        # call to probabilistic blast
        pblast.load_query_file(file)
        
        # compare with start and end index locations
        start_time = time.process_time()
        result = pblast.probabilistic_blast()
        end_time = time.process_time()
        
        total_time += end_time - start_time
        
        if start_index == result[-1][0] and result[-1][1] - result[-1][0] == length:
            exact_index_matches += 1
        elif start_index <= result[-1][0] and result[-1][1] - result[-1][0] >= length:
            contains_match += 1
        elif start_index < result[-1][1] and result[-1][1] < start_index + length:
            # contains part of the start of the query
            contains_start += 1
        elif result[-1][0] < start_index + length and start_index + length < result[-1][1]:
            # contains part of the end of the query
            contains_end += 1
        
        # check top ten results as well
        
        for j in range(-10,-1,-1):
            if start_index[1] <= result[j][0] and result[j][1] - result[j][0] >= length:
                topten += 1
            elif start_index < result[-1][1] and result[-1][1] < start_index + length:
                # contains part of the start of the query
                contains_start += 1
            elif result[-1][0] < start_index + length and start_index + length < result[-1][1]:
                # contains part of the end of the query
                contains_end += 1
    
    iterations = len(files)
    return [exact_index_matches,contains_match,contains_start,contains_end,topten/iterations,topten_contains_start/iterations,topten_contains_end/iterations,iterations,total_time/iterations]



In [23]:
pblast = blast.PBlast()

pblast.set_blast_params()

results = test_probabilistic_blast("final_testing_suite",pblast)

pblast.exit()

print(results)


total_seeds:  445
stopped before ungapped extension:  3
stopped before gapped extension:  189
HSPs:  253
BEST SCORING ALIGNMENT: 
762
1181

query: 	
HSP: 	TGGGAAAATGTGAAGGACAGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCGAACGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTGAGCCAAGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAAGGGGCAAGCCACTGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACCCTCCAGAGCAGAGCTCCCTGAAAAGATCATTCATTCACACCATACCTATTGAGCACCTACTATGTGCGTGACATTGTTCTCGGCACTGGGGGAAAGCATG
gap: 	-TGAGCAGGATGGGTAATTACGGTAGTCAGAG-------A------A--T----
db: 	
HSP: 	TGGGAAAATGTGAAGGACCGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCTAATGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTCAGCCACGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAATGGGGAAGCAACAGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACACTCCAGAGCAGAGCTCCCTGAAAAGATCATTAATTCACACAATAGCTATTGAGCACCTACTATGTGCCTGACATTGTTCTAGGCACTGGGGGAAAGCATG
gap: 	GTGAGCAGGACGGGTAATTACGGTAGTCAGAGAA

total_seeds:  324
stopped before ungapped extension:  3
stopped before gapped extension:  134
HSPs:  187
BEST SCORING ALIGNMENT: 
593545
593944

query: 	
HSP: 	CTAAATTTTAAGGCTGTGAACGAGGTCTTAATGCTAACCCCTGGGGGGGCAATCCCCGGTGGAAATGACAAGACAAGCCACTTTGTTCAAATTTGAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAACGTGGAAAGGGAGCTGTGATGCAGGCGATTTGCCCCTCAGAATAAGATAGGACCCCGCAGAGTAAGTGAGCAGAAGTACGCGCCCCTCTCCCCAGCACCACAGCATCCGGAACTTACAGCAGGATAGGCGGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGTTTGCACTTCATTGGGTGCCCTGAAAAACCAAAAAATAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA

db: 	
HSP: 	CTAAATTATAAGGCTGAGAACGAGGTCTTAATGCTAGCCCCTGTGGGGTCAATCCCCTGGGGAAATGACAAGACAAGCCACTTTGTTCAAATTTAAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAATGAGGAAAGGGAGCTGTGATGCAGGCGATTTACCCCTCAGAATAAGAGACGATCGCGCAGAGTAAGCGAGCAGAAGAACTCCGCCCCGTCCCCAGCACCCCTGCCTTTGGAACTTACAGCAGGATGGGCTGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGGTTGCACTTCATTGGTTGCCCTGAAAAACCAAAAAAGAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA
351.4799999999999
total_s

total_seeds:  461
stopped before ungapped extension:  6
stopped before gapped extension:  252
HSPs:  203
BEST SCORING ALIGNMENT: 
591205
591604

query: 	
HSP: 	AGCTCCCAGAATCCTGCTTCCCAACTAGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCTGGAGACTGCTAAATGACACCCGGAGCAACGCTGCCCTGGTATATTCAGGGGCTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGAGACCACGCTGGGAGAACAACCCGCCGGCTCTCCTAGAAGGGCTGCTCTGCTCCCCCCCCCCCACGGCCAAGGCAGGAAAATACCCCTCTGGTCAAGGAATGATGGGTCACTCACAGCAGTGGGGACAGTGACCCTTGGTGGCAGCTGGATATGTGCCTCAAAGCATCCCTGACTACTGTCCTACAGACATCTGAGGCCCGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA

db: 	
HSP: 	AGCTCCCAGAATCCTGCTCCCCAACTCGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCAGGAGACTGATAAATGACATCCGGAGCAACGCTGCCCTGGTATATTCAGGGACTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGGGACCACGCTGGGGGAACAACCCACCGGCTCGCCTGGAAGGGCTGCTCTGCTCCCCCCCCCCCACTGCCAAGGCAGGAAAATACCCCTCTGGTCATGGATGGATGGGTCACTCACAACAGTGGGGACAGTGCCCCTTGGTGGCAGCTGGATATGTGCCTCCTAGCATCCCTGACTATTGTCCTACAGACATCTGAGGCCAGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA
367.32000000000056
total_

total_seeds:  502
stopped before ungapped extension:  1
stopped before gapped extension:  265
HSPs:  236
BEST SCORING ALIGNMENT: 
39792
40191

query: 	
HSP: 	TGGCCAGACTCACCTCATTTGCAAGCCACTCGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGGGTTGGTGCCACTGGAGCCGGTGGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTGCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCGTGCCTGTTCCTTCCAACCAACAACTGGAAATGCCCACCTGGAAGTCCCTGGACTGCCCACAAGAGGCCTGGGCACCAGGGAGTTAAAACCAGACACTAGGCAGAGAGGATCGTAGAGAGCCCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTAGGGAAATACCGGCATCTGG

db: 	
HSP: 	TGGCCAGACTCACCTGATTTGCAAGCCACTGGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGAGTTGGTGCCACTGGAGCCTGTCGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTCCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCCTGCCTGTTCCTTCCAACCAACAACTGGGAATGCCCACCTGGAAGTCCTGGGGCTGCCCACAAGAGGCCTGGGCACCAGGGAGGGAAAACCAGACAGTAGGCAGAGAGGATAGTAGAGAGACCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTTGGGAAATTCAGGCATCTGG
380.39999999999975
total_se

total_seeds:  270
stopped before ungapped extension:  4
stopped before gapped extension:  184
HSPs:  82
BEST SCORING ALIGNMENT: 
197430
197829

query: 	
HSP: 	CAAACAGAGGAAACGGCTCTGAGAGAAGCCCTAGCGGACGGGAGCGGTGGCTAGGTGAGGCCGTGAGTTCAAGAACAGCCTGGACAGCTGGGCCGGGCCCGGGCTCTACTAGATCGCAGGTGTCAGCTGGTCAGGGAGGCGCTCCCCGGGAGGCAGACGAGTGGGGTTAGCTTGAGTCGGGTACATGGCAGTGGGGACGAGCAAGAGAAGGGCACAGGGACCCAGCATGGGCGACAGAAACGAGGCACGGGCCCTGGAACATGACTGTATAGGTATGAATCTCAGCTCTACCACTTTTTAGCTTAGGGGCCTTGGGCATGTTACTTAACCTCCTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCACAAAGGGTTTTATGGAAACAAAG

db: 	
HSP: 	CAAAAAGGGAAAACCTATCTGAGAGAAGCCTAACCGGACGGGGACGGTGGCACCCTGAGGCCGGGAGTTCAAGAGCAGCCTGGACAACCCGGCGGGGCCCAGTCTCTACTAGACTGCAGGAATTAGCTGGTCAGGGAGGCGCCCCCCGGGAGGCAGAAAAGAGGGATTAGCTTGAGCCGGGGAGACGGTAGTTGAGAGGAGCGAGATAATGGCACAGGACTCCGGCCTGGGCGGCAGATGTGAGGCACGGGCTCTGGAACATGACTCTCTAGGTGTGAATCTCAGCTCTACCACTTTTTAGCTGTGGGGCCTTGGGCAAGTTACTTAACCTCGTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCTCAAAGGGTAGTTTGGAAACAAAG
281.8699999999999
total_se

...

total_seeds:  409
stopped before ungapped extension:  4
stopped before gapped extension:  167
HSPs:  238
BEST SCORING ALIGNMENT: 
137793
138212

query: 	
HSP: 	CAGAAATTGATGCAGCGCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCCGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGATTCGTTACGAGGCACTCCATCCTCGGGTGATTGCTGCAGCTACAGTGACAGTATGCATTCCTCATCCGTGTGACTTTTTTCTCCTCTTCTCCCAAGACCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	-ATCCTCCCAGGCT-CAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCTGCCACTGTGCAGACCGTAGCTGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGATTCCGGGA-------------------
db: 	
HSP: 	CAGAAATTGATGCAGCCCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCTGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGAATCGTTATGAGGCACTCCATCCTCAGGTTATTGCTGCAGCTACAGTGACAGAATGCATTCCTCGTCCATGTGACTTTTTTCTCCTCTTCTCCCAAGTCCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	CATCCTCCCAGG-TGCAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCAGCCACGGTGCAGACAGTGGCCGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGA

total_seeds:  311
stopped before ungapped extension:  0
stopped before gapped extension:  184
HSPs:  127
BEST SCORING ALIGNMENT: 
443090
443489

query: 	
HSP: 	AGTAAACATTAATTAATTTCACCAGGCCTGATAAGAGTTGAGAGAAATGAAAACTGGGAGACAAACAGAAGTAGTGATATCCGAAAATAAGCTCCACCATTGCAGAAAAGCATCTCCAAGAAAATAATCTCCACCATTTAAGAAAATAACCTCCACCATTTGTTCTGCATCCCGTGGTTGGCTAATGTCCTTTTACAGCCACTGTTTCATGCGAACTCCCCGCCACCAGGAGAGTATCCTTAGCCCCATATTACAGGTGAGGACACCGAGGCCGAGAGCGGAAACCCGGCACCGAGGAGGTCCCCAGCGCGTCCGTGGACCAGGTGAGGCCAGAAGAAGCCAGGCTCGGATGCCCAACCCCGGGACATGGTTCTCTCCTTGTCCCAGGCCTGGTCCCTTG

db: 	
HSP: 	AGTAAACATTAATTAATTTCACTAGGCCTGATAAGAGTTGAGAGAAATGGAAACTGGGAGACAAACAGAAGTAGTGATATCAGAAAATAACCTCCACCATTCCAGAAAATAATCTCCAAGAAAATAATCTCCACCATTTGAGAAAATAATCTCCACCATTTGTTCTGAATCATGTGGTTTGCTAATGTCCTTTTACAGCCACTGTCTCGTGTGATCTCCCGGCCACCTGGAGAGTATCATTATCCCCATTTTACAGGTGAGGACACCGAGGCCTAGAGGGGATAAATGGCACGTGGGAGGCCACCAGCACGTCCGTGGACGAGGTGAGGCCAGAAGAATCCAGGCTCGGATGCCAAACCCCAGGACTGGGTTCTCTCCTTGTCCCTGGCCTGGTCCCTTG
335.3000000000003
total_s

total_seeds:  453
stopped before ungapped extension:  0
stopped before gapped extension:  214
HSPs:  239
BEST SCORING ALIGNMENT: 
327296
327695

query: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACCTGTCCCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCTATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAATGTTGTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGTCGCGATAAGCAATCCTAAAGGACGTGGTGGACACACCAGAGGGCGGGGCACCCAGCCCTGCCGCTGGTGTTCCCGCAGTCCACCCACCCGCGCGCTGTGCCTGTCACTTTCCTCTTTCCTCACCGCTATCGTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA

db: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACGTGTCTCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCAATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAACGTTTTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGGCGAGATAAGCAATCCTAAGGAACGTGGTGGACACGCCAGAGGGCGGGGCACCCAGCCCTGCCTCTGGTGTTCCCGCTGTCCACCCGCCCGCGCGCTCTGCCTGTCATTTTCCTCATTCCTCACCGCTATCTTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA
375.1300000000009
total_s

total_seeds:  422
stopped before ungapped extension:  6
stopped before gapped extension:  200
HSPs:  216
BEST SCORING ALIGNMENT: 
64317
64716

query: 	
HSP: 	GACCAATAGTTTCCACTCTCGCCTTCTCACCTCAGGGCCAAGCCTCCAGTAGAGATTCTGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCGCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGAATCAGACCAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGATGGGGGTGTGTGTGGGTTATGGCGAGGGTGTGTGGAGGAGGTTTGACAACAGGTAGCTTGTGGGTAGGACGTTTCTAAAGATTCCTTCCTGTTGAAGTCTTGGCTACTGTGCAATGAGGTTGTGCCTCATTC

db: 	
HSP: 	GACCAATAGGTTCCTCTCTCTCCTTCTCACCTCAGGGCCATGCCTCCAGTAGAGATTCGGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCTCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGATTCAGACAAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGTTGGGGGTGTGTGTGGGTTGTGGAGGGGGTATGTGGAGGAGGTTTGAGGACAGGTAGCTAGGGGGTAGGAAGTTTCTAAAGATTCCCTCCTGGTGAAGTCTTTGCTACTGTGCAATGAGGTTGTGTCTCATTC
373.7799999999997
total_see

total_seeds:  246
stopped before ungapped extension:  5
stopped before gapped extension:  125
HSPs:  116
BEST SCORING ALIGNMENT: 
399313
399712

query: 	
HSP: 	AGCCAAGACTAGAACTGGACAAGAGTTCAGGCTACGTAAGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTTAAAAATGTTTTAAAGGGACATATTTCATCAGGATGTCTGTCACGCCAGCGGTGGAAGCCAACACACCAATTCTCGGAGCCTGTTCCCCCTTCGCGTCAGATCCTCACCCTCAGCCCCTAGGCCCGGAGGCTGGCTGCTTAGCCCAGGGCACAGGAGGGGCCTGCGCCCTGGGCTTGGGGCTGGCATACAGCAGGTGCTCCGTGAACACGGAGGGAACGAGGATCCTGAGCTGATTCGAGAGGATGACAACGGAGAACGTTCGAACAGCTCTCCCGTGTGCTCGGCACCGTGCTGAGTGGATTACCAGCATTAACTCACGT

db: 	
HSP: 	AGCCAAGACTAGAACCTGACAACAGTTCAGGCTACGTATGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTCAAAAATGTTTTAAAGTGACATATTTCATCAGGATGTCTGTCACGCCAGCAGTGGGAGCCAACACACCAATTCTCGGAGGCTGTTCCCCCTGCGCGTCAGATCCACACGCTCAGCCCCCAGGCCCTGAGGCTGGCGGCTTAGCCCAGGGCACGGGAGGGGCCTGCTCCCTGGGCTTGGTGCTGGCACACAGCAGGTGCTCAGTAAACGCTGAGGGAAGGAGGAGGCTGAGCTGATTTGAGAGGATAACAACGGAGAACATTCCTACAGCTCTCCCGTGTGCCGGGCACCGTGCTGAGTGCATTACCTGCATTAACTCACGC
344.37000000000063
[1, 11

In [24]:
pblast = blast.PBlast()

pblast.set_blast_params(seed_stop=0.6, ungapped_stop= 10)

results = test_probabilistic_blast("final_testing_suite",pblast)

pblast.exit()

print(results)

total_seeds:  445
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  445
BEST SCORING ALIGNMENT: 
762
1181

query: 	
HSP: 	TGGGAAAATGTGAAGGACAGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCGAACGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTGAGCCAAGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAAGGGGCAAGCCACTGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACCCTCCAGAGCAGAGCTCCCTGAAAAGATCATTCATTCACACCATACCTATTGAGCACCTACTATGTGCGTGACATTGTTCTCGGCACTGGGGGAAAGCATG
gap: 	-TGAGCAGGATGGGTAATTACGGTAGTCAGAG-------A------A--T----
db: 	
HSP: 	TGGGAAAATGTGAAGGACCGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCTAATGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTCAGCCACGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAATGGGGAAGCAACAGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACACTCCAGAGCAGAGCTCCCTGAAAAGATCATTAATTCACACAATAGCTATTGAGCACCTACTATGTGCCTGACATTGTTCTAGGCACTGGGGGAAAGCATG
gap: 	GTGAGCAGGACGGGTAATTACGGTAGTCAGAGAATA

total_seeds:  324
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  324
BEST SCORING ALIGNMENT: 
593545
593944

query: 	
HSP: 	CTAAATTTTAAGGCTGTGAACGAGGTCTTAATGCTAACCCCTGGGGGGGCAATCCCCGGTGGAAATGACAAGACAAGCCACTTTGTTCAAATTTGAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAACGTGGAAAGGGAGCTGTGATGCAGGCGATTTGCCCCTCAGAATAAGATAGGACCCCGCAGAGTAAGTGAGCAGAAGTACGCGCCCCTCTCCCCAGCACCACAGCATCCGGAACTTACAGCAGGATAGGCGGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGTTTGCACTTCATTGGGTGCCCTGAAAAACCAAAAAATAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA

db: 	
HSP: 	CTAAATTATAAGGCTGAGAACGAGGTCTTAATGCTAGCCCCTGTGGGGTCAATCCCCTGGGGAAATGACAAGACAAGCCACTTTGTTCAAATTTAAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAATGAGGAAAGGGAGCTGTGATGCAGGCGATTTACCCCTCAGAATAAGAGACGATCGCGCAGAGTAAGCGAGCAGAAGAACTCCGCCCCGTCCCCAGCACCCCTGCCTTTGGAACTTACAGCAGGATGGGCTGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGGTTGCACTTCATTGGTTGCCCTGAAAAACCAAAAAAGAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA
351.4799999999999
total_see

total_seeds:  461
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  461
BEST SCORING ALIGNMENT: 
591205
591604

query: 	
HSP: 	AGCTCCCAGAATCCTGCTTCCCAACTAGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCTGGAGACTGCTAAATGACACCCGGAGCAACGCTGCCCTGGTATATTCAGGGGCTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGAGACCACGCTGGGAGAACAACCCGCCGGCTCTCCTAGAAGGGCTGCTCTGCTCCCCCCCCCCCACGGCCAAGGCAGGAAAATACCCCTCTGGTCAAGGAATGATGGGTCACTCACAGCAGTGGGGACAGTGACCCTTGGTGGCAGCTGGATATGTGCCTCAAAGCATCCCTGACTACTGTCCTACAGACATCTGAGGCCCGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA

db: 	
HSP: 	AGCTCCCAGAATCCTGCTCCCCAACTCGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCAGGAGACTGATAAATGACATCCGGAGCAACGCTGCCCTGGTATATTCAGGGACTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGGGACCACGCTGGGGGAACAACCCACCGGCTCGCCTGGAAGGGCTGCTCTGCTCCCCCCCCCCCACTGCCAAGGCAGGAAAATACCCCTCTGGTCATGGATGGATGGGTCACTCACAACAGTGGGGACAGTGCCCCTTGGTGGCAGCTGGATATGTGCCTCCTAGCATCCCTGACTATTGTCCTACAGACATCTGAGGCCAGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA
367.32000000000056
total_se

total_seeds:  502
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  502
BEST SCORING ALIGNMENT: 
39792
40191

query: 	
HSP: 	TGGCCAGACTCACCTCATTTGCAAGCCACTCGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGGGTTGGTGCCACTGGAGCCGGTGGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTGCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCGTGCCTGTTCCTTCCAACCAACAACTGGAAATGCCCACCTGGAAGTCCCTGGACTGCCCACAAGAGGCCTGGGCACCAGGGAGTTAAAACCAGACACTAGGCAGAGAGGATCGTAGAGAGCCCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTAGGGAAATACCGGCATCTGG

db: 	
HSP: 	TGGCCAGACTCACCTGATTTGCAAGCCACTGGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGAGTTGGTGCCACTGGAGCCTGTCGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTCCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCCTGCCTGTTCCTTCCAACCAACAACTGGGAATGCCCACCTGGAAGTCCTGGGGCTGCCCACAAGAGGCCTGGGCACCAGGGAGGGAAAACCAGACAGTAGGCAGAGAGGATAGTAGAGAGACCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTTGGGAAATTCAGGCATCTGG
380.39999999999975
total_seed

total_seeds:  270
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  270
BEST SCORING ALIGNMENT: 
197430
197829

query: 	
HSP: 	CAAACAGAGGAAACGGCTCTGAGAGAAGCCCTAGCGGACGGGAGCGGTGGCTAGGTGAGGCCGTGAGTTCAAGAACAGCCTGGACAGCTGGGCCGGGCCCGGGCTCTACTAGATCGCAGGTGTCAGCTGGTCAGGGAGGCGCTCCCCGGGAGGCAGACGAGTGGGGTTAGCTTGAGTCGGGTACATGGCAGTGGGGACGAGCAAGAGAAGGGCACAGGGACCCAGCATGGGCGACAGAAACGAGGCACGGGCCCTGGAACATGACTGTATAGGTATGAATCTCAGCTCTACCACTTTTTAGCTTAGGGGCCTTGGGCATGTTACTTAACCTCCTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCACAAAGGGTTTTATGGAAACAAAG

db: 	
HSP: 	CAAAAAGGGAAAACCTATCTGAGAGAAGCCTAACCGGACGGGGACGGTGGCACCCTGAGGCCGGGAGTTCAAGAGCAGCCTGGACAACCCGGCGGGGCCCAGTCTCTACTAGACTGCAGGAATTAGCTGGTCAGGGAGGCGCCCCCCGGGAGGCAGAAAAGAGGGATTAGCTTGAGCCGGGGAGACGGTAGTTGAGAGGAGCGAGATAATGGCACAGGACTCCGGCCTGGGCGGCAGATGTGAGGCACGGGCTCTGGAACATGACTCTCTAGGTGTGAATCTCAGCTCTACCACTTTTTAGCTGTGGGGCCTTGGGCAAGTTACTTAACCTCGTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCTCAAAGGGTAGTTTGGAAACAAAG
281.8699999999999
total_see

...

total_seeds:  409
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  409
BEST SCORING ALIGNMENT: 
137793
138212

query: 	
HSP: 	CAGAAATTGATGCAGCGCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCCGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGATTCGTTACGAGGCACTCCATCCTCGGGTGATTGCTGCAGCTACAGTGACAGTATGCATTCCTCATCCGTGTGACTTTTTTCTCCTCTTCTCCCAAGACCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	-ATCCTCCCAGGCT-CAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCTGCCACTGTGCAGACCGTAGCTGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGATTCCGGGA-------------------
db: 	
HSP: 	CAGAAATTGATGCAGCCCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCTGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGAATCGTTATGAGGCACTCCATCCTCAGGTTATTGCTGCAGCTACAGTGACAGAATGCATTCCTCGTCCATGTGACTTTTTTCTCCTCTTCTCCCAAGTCCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	CATCCTCCCAGG-TGCAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCAGCCACGGTGCAGACAGTGGCCGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGATT

total_seeds:  311
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  311
BEST SCORING ALIGNMENT: 
443090
443489

query: 	
HSP: 	AGTAAACATTAATTAATTTCACCAGGCCTGATAAGAGTTGAGAGAAATGAAAACTGGGAGACAAACAGAAGTAGTGATATCCGAAAATAAGCTCCACCATTGCAGAAAAGCATCTCCAAGAAAATAATCTCCACCATTTAAGAAAATAACCTCCACCATTTGTTCTGCATCCCGTGGTTGGCTAATGTCCTTTTACAGCCACTGTTTCATGCGAACTCCCCGCCACCAGGAGAGTATCCTTAGCCCCATATTACAGGTGAGGACACCGAGGCCGAGAGCGGAAACCCGGCACCGAGGAGGTCCCCAGCGCGTCCGTGGACCAGGTGAGGCCAGAAGAAGCCAGGCTCGGATGCCCAACCCCGGGACATGGTTCTCTCCTTGTCCCAGGCCTGGTCCCTTG

db: 	
HSP: 	AGTAAACATTAATTAATTTCACTAGGCCTGATAAGAGTTGAGAGAAATGGAAACTGGGAGACAAACAGAAGTAGTGATATCAGAAAATAACCTCCACCATTCCAGAAAATAATCTCCAAGAAAATAATCTCCACCATTTGAGAAAATAATCTCCACCATTTGTTCTGAATCATGTGGTTTGCTAATGTCCTTTTACAGCCACTGTCTCGTGTGATCTCCCGGCCACCTGGAGAGTATCATTATCCCCATTTTACAGGTGAGGACACCGAGGCCTAGAGGGGATAAATGGCACGTGGGAGGCCACCAGCACGTCCGTGGACGAGGTGAGGCCAGAAGAATCCAGGCTCGGATGCCAAACCCCAGGACTGGGTTCTCTCCTTGTCCCTGGCCTGGTCCCTTG
335.3000000000003
total_see

total_seeds:  453
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  453
BEST SCORING ALIGNMENT: 
327296
327695

query: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACCTGTCCCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCTATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAATGTTGTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGTCGCGATAAGCAATCCTAAAGGACGTGGTGGACACACCAGAGGGCGGGGCACCCAGCCCTGCCGCTGGTGTTCCCGCAGTCCACCCACCCGCGCGCTGTGCCTGTCACTTTCCTCTTTCCTCACCGCTATCGTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA

db: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACGTGTCTCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCAATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAACGTTTTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGGCGAGATAAGCAATCCTAAGGAACGTGGTGGACACGCCAGAGGGCGGGGCACCCAGCCCTGCCTCTGGTGTTCCCGCTGTCCACCCGCCCGCGCGCTCTGCCTGTCATTTTCCTCATTCCTCACCGCTATCTTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA
375.1300000000009
total_see

total_seeds:  422
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  422
BEST SCORING ALIGNMENT: 
64317
64716

query: 	
HSP: 	GACCAATAGTTTCCACTCTCGCCTTCTCACCTCAGGGCCAAGCCTCCAGTAGAGATTCTGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCGCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGAATCAGACCAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGATGGGGGTGTGTGTGGGTTATGGCGAGGGTGTGTGGAGGAGGTTTGACAACAGGTAGCTTGTGGGTAGGACGTTTCTAAAGATTCCTTCCTGTTGAAGTCTTGGCTACTGTGCAATGAGGTTGTGCCTCATTC

db: 	
HSP: 	GACCAATAGGTTCCTCTCTCTCCTTCTCACCTCAGGGCCATGCCTCCAGTAGAGATTCGGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCTCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGATTCAGACAAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGTTGGGGGTGTGTGTGGGTTGTGGAGGGGGTATGTGGAGGAGGTTTGAGGACAGGTAGCTAGGGGGTAGGAAGTTTCTAAAGATTCCCTCCTGGTGAAGTCTTTGCTACTGTGCAATGAGGTTGTGTCTCATTC
373.7799999999997
total_seeds

total_seeds:  246
stopped before ungapped extension:  0
stopped before gapped extension:  0
HSPs:  246
BEST SCORING ALIGNMENT: 
399313
399712

query: 	
HSP: 	AGCCAAGACTAGAACTGGACAAGAGTTCAGGCTACGTAAGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTTAAAAATGTTTTAAAGGGACATATTTCATCAGGATGTCTGTCACGCCAGCGGTGGAAGCCAACACACCAATTCTCGGAGCCTGTTCCCCCTTCGCGTCAGATCCTCACCCTCAGCCCCTAGGCCCGGAGGCTGGCTGCTTAGCCCAGGGCACAGGAGGGGCCTGCGCCCTGGGCTTGGGGCTGGCATACAGCAGGTGCTCCGTGAACACGGAGGGAACGAGGATCCTGAGCTGATTCGAGAGGATGACAACGGAGAACGTTCGAACAGCTCTCCCGTGTGCTCGGCACCGTGCTGAGTGGATTACCAGCATTAACTCACGT

db: 	
HSP: 	AGCCAAGACTAGAACCTGACAACAGTTCAGGCTACGTATGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTCAAAAATGTTTTAAAGTGACATATTTCATCAGGATGTCTGTCACGCCAGCAGTGGGAGCCAACACACCAATTCTCGGAGGCTGTTCCCCCTGCGCGTCAGATCCACACGCTCAGCCCCCAGGCCCTGAGGCTGGCGGCTTAGCCCAGGGCACGGGAGGGGCCTGCTCCCTGGGCTTGGTGCTGGCACACAGCAGGTGCTCAGTAAACGCTGAGGGAAGGAGGAGGCTGAGCTGATTTGAGAGGATAACAACGGAGAACATTCCTACAGCTCTCCCGTGTGCCGGGCACCGTGCTGAGTGCATTACCTGCATTAACTCACGC
344.37000000000063
[1, 10, 

In [25]:
pblast = blast.PBlast()

pblast.set_blast_params(max_decrease_stop=20)

results = test_probabilistic_blast("final_testing_suite",pblast)

pblast.exit()

print(results)


total_seeds:  445
stopped before ungapped extension:  3
stopped before gapped extension:  189
HSPs:  253
BEST SCORING ALIGNMENT: 
762
1181

query: 	
HSP: 	TGGGAAAATGTGAAGGACAGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCGAACGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTGAGCCAAGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAAGGGGCAAGCCACTGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACCCTCCAGAGCAGAGCTCCCTGAAAAGATCATTCATTCACACCATACCTATTGAGCACCTACTATGTGCGTGACATTGTTCTCGGCACTGGGGGAAAGCATG
gap: 	-TGAGCAGGATGGGTAATTACGGTAGTCAGAG-------A------A--T----
db: 	
HSP: 	TGGGAAAATGTGAAGGACCGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCTAATGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTCAGCCACGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAATGGGGAAGCAACAGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACACTCCAGAGCAGAGCTCCCTGAAAAGATCATTAATTCACACAATAGCTATTGAGCACCTACTATGTGCCTGACATTGTTCTAGGCACTGGGGGAAAGCATG
gap: 	GTGAGCAGGACGGGTAATTACGGTAGTCAGAGAA

total_seeds:  324
stopped before ungapped extension:  3
stopped before gapped extension:  134
HSPs:  187
BEST SCORING ALIGNMENT: 
593545
593944

query: 	
HSP: 	CTAAATTTTAAGGCTGTGAACGAGGTCTTAATGCTAACCCCTGGGGGGGCAATCCCCGGTGGAAATGACAAGACAAGCCACTTTGTTCAAATTTGAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAACGTGGAAAGGGAGCTGTGATGCAGGCGATTTGCCCCTCAGAATAAGATAGGACCCCGCAGAGTAAGTGAGCAGAAGTACGCGCCCCTCTCCCCAGCACCACAGCATCCGGAACTTACAGCAGGATAGGCGGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGTTTGCACTTCATTGGGTGCCCTGAAAAACCAAAAAATAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA

db: 	
HSP: 	CTAAATTATAAGGCTGAGAACGAGGTCTTAATGCTAGCCCCTGTGGGGTCAATCCCCTGGGGAAATGACAAGACAAGCCACTTTGTTCAAATTTAAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAATGAGGAAAGGGAGCTGTGATGCAGGCGATTTACCCCTCAGAATAAGAGACGATCGCGCAGAGTAAGCGAGCAGAAGAACTCCGCCCCGTCCCCAGCACCCCTGCCTTTGGAACTTACAGCAGGATGGGCTGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGGTTGCACTTCATTGGTTGCCCTGAAAAACCAAAAAAGAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA
351.4799999999999
total_s

total_seeds:  461
stopped before ungapped extension:  6
stopped before gapped extension:  252
HSPs:  203
BEST SCORING ALIGNMENT: 
591205
591604

query: 	
HSP: 	AGCTCCCAGAATCCTGCTTCCCAACTAGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCTGGAGACTGCTAAATGACACCCGGAGCAACGCTGCCCTGGTATATTCAGGGGCTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGAGACCACGCTGGGAGAACAACCCGCCGGCTCTCCTAGAAGGGCTGCTCTGCTCCCCCCCCCCCACGGCCAAGGCAGGAAAATACCCCTCTGGTCAAGGAATGATGGGTCACTCACAGCAGTGGGGACAGTGACCCTTGGTGGCAGCTGGATATGTGCCTCAAAGCATCCCTGACTACTGTCCTACAGACATCTGAGGCCCGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA

db: 	
HSP: 	AGCTCCCAGAATCCTGCTCCCCAACTCGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCAGGAGACTGATAAATGACATCCGGAGCAACGCTGCCCTGGTATATTCAGGGACTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGGGACCACGCTGGGGGAACAACCCACCGGCTCGCCTGGAAGGGCTGCTCTGCTCCCCCCCCCCCACTGCCAAGGCAGGAAAATACCCCTCTGGTCATGGATGGATGGGTCACTCACAACAGTGGGGACAGTGCCCCTTGGTGGCAGCTGGATATGTGCCTCCTAGCATCCCTGACTATTGTCCTACAGACATCTGAGGCCAGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA
367.32000000000056
total_

total_seeds:  502
stopped before ungapped extension:  1
stopped before gapped extension:  265
HSPs:  236
BEST SCORING ALIGNMENT: 
39792
40191

query: 	
HSP: 	TGGCCAGACTCACCTCATTTGCAAGCCACTCGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGGGTTGGTGCCACTGGAGCCGGTGGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTGCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCGTGCCTGTTCCTTCCAACCAACAACTGGAAATGCCCACCTGGAAGTCCCTGGACTGCCCACAAGAGGCCTGGGCACCAGGGAGTTAAAACCAGACACTAGGCAGAGAGGATCGTAGAGAGCCCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTAGGGAAATACCGGCATCTGG

db: 	
HSP: 	TGGCCAGACTCACCTGATTTGCAAGCCACTGGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGAGTTGGTGCCACTGGAGCCTGTCGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTCCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCCTGCCTGTTCCTTCCAACCAACAACTGGGAATGCCCACCTGGAAGTCCTGGGGCTGCCCACAAGAGGCCTGGGCACCAGGGAGGGAAAACCAGACAGTAGGCAGAGAGGATAGTAGAGAGACCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTTGGGAAATTCAGGCATCTGG
380.39999999999975
total_se

total_seeds:  270
stopped before ungapped extension:  4
stopped before gapped extension:  184
HSPs:  82
BEST SCORING ALIGNMENT: 
197430
197829

query: 	
HSP: 	CAAACAGAGGAAACGGCTCTGAGAGAAGCCCTAGCGGACGGGAGCGGTGGCTAGGTGAGGCCGTGAGTTCAAGAACAGCCTGGACAGCTGGGCCGGGCCCGGGCTCTACTAGATCGCAGGTGTCAGCTGGTCAGGGAGGCGCTCCCCGGGAGGCAGACGAGTGGGGTTAGCTTGAGTCGGGTACATGGCAGTGGGGACGAGCAAGAGAAGGGCACAGGGACCCAGCATGGGCGACAGAAACGAGGCACGGGCCCTGGAACATGACTGTATAGGTATGAATCTCAGCTCTACCACTTTTTAGCTTAGGGGCCTTGGGCATGTTACTTAACCTCCTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCACAAAGGGTTTTATGGAAACAAAG

db: 	
HSP: 	CAAAAAGGGAAAACCTATCTGAGAGAAGCCTAACCGGACGGGGACGGTGGCACCCTGAGGCCGGGAGTTCAAGAGCAGCCTGGACAACCCGGCGGGGCCCAGTCTCTACTAGACTGCAGGAATTAGCTGGTCAGGGAGGCGCCCCCCGGGAGGCAGAAAAGAGGGATTAGCTTGAGCCGGGGAGACGGTAGTTGAGAGGAGCGAGATAATGGCACAGGACTCCGGCCTGGGCGGCAGATGTGAGGCACGGGCTCTGGAACATGACTCTCTAGGTGTGAATCTCAGCTCTACCACTTTTTAGCTGTGGGGCCTTGGGCAAGTTACTTAACCTCGTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCTCAAAGGGTAGTTTGGAAACAAAG
281.8699999999999
total_se

...

total_seeds:  409
stopped before ungapped extension:  4
stopped before gapped extension:  167
HSPs:  238
BEST SCORING ALIGNMENT: 
137793
138212

query: 	
HSP: 	CAGAAATTGATGCAGCGCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCCGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGATTCGTTACGAGGCACTCCATCCTCGGGTGATTGCTGCAGCTACAGTGACAGTATGCATTCCTCATCCGTGTGACTTTTTTCTCCTCTTCTCCCAAGACCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	-ATCCTCCCAGGCT-CAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCTGCCACTGTGCAGACCGTAGCTGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGATTCCGGGA-------------------
db: 	
HSP: 	CAGAAATTGATGCAGCCCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCTGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGAATCGTTATGAGGCACTCCATCCTCAGGTTATTGCTGCAGCTACAGTGACAGAATGCATTCCTCGTCCATGTGACTTTTTTCTCCTCTTCTCCCAAGTCCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	CATCCTCCCAGG-TGCAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCAGCCACGGTGCAGACAGTGGCCGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGA

total_seeds:  311
stopped before ungapped extension:  0
stopped before gapped extension:  184
HSPs:  127
BEST SCORING ALIGNMENT: 
443090
443489

query: 	
HSP: 	AGTAAACATTAATTAATTTCACCAGGCCTGATAAGAGTTGAGAGAAATGAAAACTGGGAGACAAACAGAAGTAGTGATATCCGAAAATAAGCTCCACCATTGCAGAAAAGCATCTCCAAGAAAATAATCTCCACCATTTAAGAAAATAACCTCCACCATTTGTTCTGCATCCCGTGGTTGGCTAATGTCCTTTTACAGCCACTGTTTCATGCGAACTCCCCGCCACCAGGAGAGTATCCTTAGCCCCATATTACAGGTGAGGACACCGAGGCCGAGAGCGGAAACCCGGCACCGAGGAGGTCCCCAGCGCGTCCGTGGACCAGGTGAGGCCAGAAGAAGCCAGGCTCGGATGCCCAACCCCGGGACATGGTTCTCTCCTTGTCCCAGGCCTGGTCCCTTG

db: 	
HSP: 	AGTAAACATTAATTAATTTCACTAGGCCTGATAAGAGTTGAGAGAAATGGAAACTGGGAGACAAACAGAAGTAGTGATATCAGAAAATAACCTCCACCATTCCAGAAAATAATCTCCAAGAAAATAATCTCCACCATTTGAGAAAATAATCTCCACCATTTGTTCTGAATCATGTGGTTTGCTAATGTCCTTTTACAGCCACTGTCTCGTGTGATCTCCCGGCCACCTGGAGAGTATCATTATCCCCATTTTACAGGTGAGGACACCGAGGCCTAGAGGGGATAAATGGCACGTGGGAGGCCACCAGCACGTCCGTGGACGAGGTGAGGCCAGAAGAATCCAGGCTCGGATGCCAAACCCCAGGACTGGGTTCTCTCCTTGTCCCTGGCCTGGTCCCTTG
335.3000000000003
total_s

total_seeds:  453
stopped before ungapped extension:  0
stopped before gapped extension:  214
HSPs:  239
BEST SCORING ALIGNMENT: 
327296
327695

query: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACCTGTCCCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCTATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAATGTTGTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGTCGCGATAAGCAATCCTAAAGGACGTGGTGGACACACCAGAGGGCGGGGCACCCAGCCCTGCCGCTGGTGTTCCCGCAGTCCACCCACCCGCGCGCTGTGCCTGTCACTTTCCTCTTTCCTCACCGCTATCGTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA

db: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACGTGTCTCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCAATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAACGTTTTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGGCGAGATAAGCAATCCTAAGGAACGTGGTGGACACGCCAGAGGGCGGGGCACCCAGCCCTGCCTCTGGTGTTCCCGCTGTCCACCCGCCCGCGCGCTCTGCCTGTCATTTTCCTCATTCCTCACCGCTATCTTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA
375.1300000000009
total_s

total_seeds:  422
stopped before ungapped extension:  6
stopped before gapped extension:  200
HSPs:  216
BEST SCORING ALIGNMENT: 
64317
64716

query: 	
HSP: 	GACCAATAGTTTCCACTCTCGCCTTCTCACCTCAGGGCCAAGCCTCCAGTAGAGATTCTGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCGCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGAATCAGACCAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGATGGGGGTGTGTGTGGGTTATGGCGAGGGTGTGTGGAGGAGGTTTGACAACAGGTAGCTTGTGGGTAGGACGTTTCTAAAGATTCCTTCCTGTTGAAGTCTTGGCTACTGTGCAATGAGGTTGTGCCTCATTC

db: 	
HSP: 	GACCAATAGGTTCCTCTCTCTCCTTCTCACCTCAGGGCCATGCCTCCAGTAGAGATTCGGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCTCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGATTCAGACAAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGTTGGGGGTGTGTGTGGGTTGTGGAGGGGGTATGTGGAGGAGGTTTGAGGACAGGTAGCTAGGGGGTAGGAAGTTTCTAAAGATTCCCTCCTGGTGAAGTCTTTGCTACTGTGCAATGAGGTTGTGTCTCATTC
373.7799999999997
total_see

total_seeds:  246
stopped before ungapped extension:  5
stopped before gapped extension:  125
HSPs:  116
BEST SCORING ALIGNMENT: 
399313
399712

query: 	
HSP: 	AGCCAAGACTAGAACTGGACAAGAGTTCAGGCTACGTAAGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTTAAAAATGTTTTAAAGGGACATATTTCATCAGGATGTCTGTCACGCCAGCGGTGGAAGCCAACACACCAATTCTCGGAGCCTGTTCCCCCTTCGCGTCAGATCCTCACCCTCAGCCCCTAGGCCCGGAGGCTGGCTGCTTAGCCCAGGGCACAGGAGGGGCCTGCGCCCTGGGCTTGGGGCTGGCATACAGCAGGTGCTCCGTGAACACGGAGGGAACGAGGATCCTGAGCTGATTCGAGAGGATGACAACGGAGAACGTTCGAACAGCTCTCCCGTGTGCTCGGCACCGTGCTGAGTGGATTACCAGCATTAACTCACGT

db: 	
HSP: 	AGCCAAGACTAGAACCTGACAACAGTTCAGGCTACGTATGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTCAAAAATGTTTTAAAGTGACATATTTCATCAGGATGTCTGTCACGCCAGCAGTGGGAGCCAACACACCAATTCTCGGAGGCTGTTCCCCCTGCGCGTCAGATCCACACGCTCAGCCCCCAGGCCCTGAGGCTGGCGGCTTAGCCCAGGGCACGGGAGGGGCCTGCTCCCTGGGCTTGGTGCTGGCACACAGCAGGTGCTCAGTAAACGCTGAGGGAAGGAGGAGGCTGAGCTGATTTGAGAGGATAACAACGGAGAACATTCCTACAGCTCTCCCGTGTGCCGGGCACCGTGCTGAGTGCATTACCTGCATTAACTCACGC
344.37000000000063
[1, 11

In [26]:
pblast = blast.PBlast()

pblast.set_blast_params(max_decrease_stop=4)

results = test_probabilistic_blast("final_testing_suite",pblast)

pblast.exit()

print(results)


total_seeds:  445
stopped before ungapped extension:  3
stopped before gapped extension:  189
HSPs:  253
BEST SCORING ALIGNMENT: 
762
1181

query: 	
HSP: 	TGGGAAAATGTGAAGGACAGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCGAACGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTGAGCCAAGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAAGGGGCAAGCCACTGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACCCTCCAGAGCAGAGCTCCCTGAAAAGATCATTCATTCACACCATACCTATTGAGCACCTACTATGTGCGTGACATTGTTCTCGGCACTGGGGGAAAGCATG
gap: 	-TGAGCAGGATGGGTAATTACGGTAGTCAGAG-------A------A--T----
db: 	
HSP: 	TGGGAAAATGTGAAGGACCGTGAGAAATCTGGCAGGCTGGCAAATATTTCCACAATTCAGTTCTTTTTTGATCTGTGAGGCTAATCTAATGATGGTGATAGGGCAAGGGAGTTAATGGCCCCTGAAAGGCCTACAGAAAACCCTCAGCCACGGGGTTAACAGACTGCCACAGGAGACTAAAGGAGAGACTGAGGCCCAGTGGGGAAACCAGCCAATGGGGAAGCAACAGGATCACTGGGGAGGACAGATCTGGGAAGGGAGAACACTCCAGAGCAGAGCTCCCTGAAAAGATCATTAATTCACACAATAGCTATTGAGCACCTACTATGTGCCTGACATTGTTCTAGGCACTGGGGGAAAGCATG
gap: 	GTGAGCAGGACGGGTAATTACGGTAGTCAGAGAA

total_seeds:  324
stopped before ungapped extension:  3
stopped before gapped extension:  134
HSPs:  187
BEST SCORING ALIGNMENT: 
593545
593944

query: 	
HSP: 	CTAAATTTTAAGGCTGTGAACGAGGTCTTAATGCTAACCCCTGGGGGGGCAATCCCCGGTGGAAATGACAAGACAAGCCACTTTGTTCAAATTTGAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAACGTGGAAAGGGAGCTGTGATGCAGGCGATTTGCCCCTCAGAATAAGATAGGACCCCGCAGAGTAAGTGAGCAGAAGTACGCGCCCCTCTCCCCAGCACCACAGCATCCGGAACTTACAGCAGGATAGGCGGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGTTTGCACTTCATTGGGTGCCCTGAAAAACCAAAAAATAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA

db: 	
HSP: 	CTAAATTATAAGGCTGAGAACGAGGTCTTAATGCTAGCCCCTGTGGGGTCAATCCCCTGGGGAAATGACAAGACAAGCCACTTTGTTCAAATTTAAGCATCCCGACTGACCGCCTGCCATCAGAACAGAGGAAATGGAATGAGGAAAGGGAGCTGTGATGCAGGCGATTTACCCCTCAGAATAAGAGACGATCGCGCAGAGTAAGCGAGCAGAAGAACTCCGCCCCGTCCCCAGCACCCCTGCCTTTGGAACTTACAGCAGGATGGGCTGGTCTGAGGTGATAACGTTCCCATTCATGTGAAGGTTGCACTTCATTGGTTGCCCTGAAAAACCAAAAAAGAAAAGGCTCTGCCATGAGACCTCAGCTCCTCACACAAAAGGACAAGGGCTGCCACCTCCA
351.4799999999999
total_s

total_seeds:  461
stopped before ungapped extension:  6
stopped before gapped extension:  252
HSPs:  203
BEST SCORING ALIGNMENT: 
591205
591604

query: 	
HSP: 	AGCTCCCAGAATCCTGCTTCCCAACTAGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCTGGAGACTGCTAAATGACACCCGGAGCAACGCTGCCCTGGTATATTCAGGGGCTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGAGACCACGCTGGGAGAACAACCCGCCGGCTCTCCTAGAAGGGCTGCTCTGCTCCCCCCCCCCCACGGCCAAGGCAGGAAAATACCCCTCTGGTCAAGGAATGATGGGTCACTCACAGCAGTGGGGACAGTGACCCTTGGTGGCAGCTGGATATGTGCCTCAAAGCATCCCTGACTACTGTCCTACAGACATCTGAGGCCCGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA

db: 	
HSP: 	AGCTCCCAGAATCCTGCTCCCCAACTCGCTCCAGCTCTGCCAGCCTGGATTCCCTTTACAGTCCCAACAAACCAGCAGGAGACTGATAAATGACATCCGGAGCAACGCTGCCCTGGTATATTCAGGGACTGCCCCTGGGTTCTACAGTTTTGCCCCAACTGCCTGGGACCACGCTGGGGGAACAACCCACCGGCTCGCCTGGAAGGGCTGCTCTGCTCCCCCCCCCCCACTGCCAAGGCAGGAAAATACCCCTCTGGTCATGGATGGATGGGTCACTCACAACAGTGGGGACAGTGCCCCTTGGTGGCAGCTGGATATGTGCCTCCTAGCATCCCTGACTATTGTCCTACAGACATCTGAGGCCAGAAAGAAAGAGCCAGGAGCAAGCCAACATCTCCCA
367.32000000000056
total_

total_seeds:  502
stopped before ungapped extension:  1
stopped before gapped extension:  269
HSPs:  232
BEST SCORING ALIGNMENT: 
39792
40191

query: 	
HSP: 	TGGCCAGACTCACCTCATTTGCAAGCCACTCGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGGGTTGGTGCCACTGGAGCCGGTGGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTGCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCGTGCCTGTTCCTTCCAACCAACAACTGGAAATGCCCACCTGGAAGTCCCTGGACTGCCCACAAGAGGCCTGGGCACCAGGGAGTTAAAACCAGACACTAGGCAGAGAGGATCGTAGAGAGCCCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTAGGGAAATACCGGCATCTGG

db: 	
HSP: 	TGGCCAGACTCACCTGATTTGCAAGCCACTGGCCCCCTGCCCTGGACATGTGCCAAAGGATGGAGCAGCTGCTGGTCCACTCCCAGGAAGAGTTGGTGCCACTGGAGCCTGTCGTCAGCATGAGGTCCCACCCCACCTCCATGACCTTAGCCACCTCTCTCCCAAACCTCCCTTCAGCTCAGAGGCTGCAGTTCTGCTCCAGAGAGTTCCTGCCTGTTCCTTCCAACCAACAACTGGGAATGCCCACCTGGAAGTCCTGGGGCTGCCCACAAGAGGCCTGGGCACCAGGGAGGGAAAACCAGACAGTAGGCAGAGAGGATAGTAGAGAGACCCAGGCTCCAGGGTGGGCGAACCAGAGAGAGAGCAGAGGGGAGGATGCTTGGGAAATTCAGGCATCTGG
380.39999999999975
total_se

total_seeds:  270
stopped before ungapped extension:  4
stopped before gapped extension:  184
HSPs:  82
BEST SCORING ALIGNMENT: 
197430
197829

query: 	
HSP: 	CAAACAGAGGAAACGGCTCTGAGAGAAGCCCTAGCGGACGGGAGCGGTGGCTAGGTGAGGCCGTGAGTTCAAGAACAGCCTGGACAGCTGGGCCGGGCCCGGGCTCTACTAGATCGCAGGTGTCAGCTGGTCAGGGAGGCGCTCCCCGGGAGGCAGACGAGTGGGGTTAGCTTGAGTCGGGTACATGGCAGTGGGGACGAGCAAGAGAAGGGCACAGGGACCCAGCATGGGCGACAGAAACGAGGCACGGGCCCTGGAACATGACTGTATAGGTATGAATCTCAGCTCTACCACTTTTTAGCTTAGGGGCCTTGGGCATGTTACTTAACCTCCTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCACAAAGGGTTTTATGGAAACAAAG

db: 	
HSP: 	CAAAAAGGGAAAACCTATCTGAGAGAAGCCTAACCGGACGGGGACGGTGGCACCCTGAGGCCGGGAGTTCAAGAGCAGCCTGGACAACCCGGCGGGGCCCAGTCTCTACTAGACTGCAGGAATTAGCTGGTCAGGGAGGCGCCCCCCGGGAGGCAGAAAAGAGGGATTAGCTTGAGCCGGGGAGACGGTAGTTGAGAGGAGCGAGATAATGGCACAGGACTCCGGCCTGGGCGGCAGATGTGAGGCACGGGCTCTGGAACATGACTCTCTAGGTGTGAATCTCAGCTCTACCACTTTTTAGCTGTGGGGCCTTGGGCAAGTTACTTAACCTCGTGACAGTTTCCTCAGCTATAAAACAGAGATAATAATACCTACCTCAAAGGGTAGTTTGGAAACAAAG
281.8699999999999
total_se

...

total_seeds:  409
stopped before ungapped extension:  4
stopped before gapped extension:  167
HSPs:  238
BEST SCORING ALIGNMENT: 
137793
138212

query: 	
HSP: 	CAGAAATTGATGCAGCGCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCCGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGATTCGTTACGAGGCACTCCATCCTCGGGTGATTGCTGCAGCTACAGTGACAGTATGCATTCCTCATCCGTGTGACTTTTTTCTCCTCTTCTCCCAAGACCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	-ATCCTCCCAGGCT-CAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCTGCCACTGTGCAGACCGTAGCTGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGATTCCGGGA-------------------
db: 	
HSP: 	CAGAAATTGATGCAGCCCTTATGGGATGTTCTTGTTATCTAGACCAGCTTGTAAATGGCTGTGCACATCTTCTTCAGCAGCAGGTGGGGAGTTGGAATCGTTATGAGGCACTCCATCCTCAGGTTATTGCTGCAGCTACAGTGACAGAATGCATTCCTCGTCCATGTGACTTTTTTCTCCTCTTCTCCCAAGTCCTTCAGCCTCAAGTCCAAAGCCTGGTGA
gap: 	CATCCTCCCAGG-TGCAGCCGGTCACCATCCAGCAGCAGGTGCAGACGGTGCAGGCCCAGCGGGTGCTGACGCAGACGGCCAATGGCACGCTGCAGACCCTCGCCCCAGCCACGGTGCAGACAGTGGCCGCGCCCCAGGTGCAGCAGGTCCCGGTAGGTGGCTGGCAAGGA

total_seeds:  311
stopped before ungapped extension:  0
stopped before gapped extension:  184
HSPs:  127
BEST SCORING ALIGNMENT: 
443090
443489

query: 	
HSP: 	AGTAAACATTAATTAATTTCACCAGGCCTGATAAGAGTTGAGAGAAATGAAAACTGGGAGACAAACAGAAGTAGTGATATCCGAAAATAAGCTCCACCATTGCAGAAAAGCATCTCCAAGAAAATAATCTCCACCATTTAAGAAAATAACCTCCACCATTTGTTCTGCATCCCGTGGTTGGCTAATGTCCTTTTACAGCCACTGTTTCATGCGAACTCCCCGCCACCAGGAGAGTATCCTTAGCCCCATATTACAGGTGAGGACACCGAGGCCGAGAGCGGAAACCCGGCACCGAGGAGGTCCCCAGCGCGTCCGTGGACCAGGTGAGGCCAGAAGAAGCCAGGCTCGGATGCCCAACCCCGGGACATGGTTCTCTCCTTGTCCCAGGCCTGGTCCCTTG

db: 	
HSP: 	AGTAAACATTAATTAATTTCACTAGGCCTGATAAGAGTTGAGAGAAATGGAAACTGGGAGACAAACAGAAGTAGTGATATCAGAAAATAACCTCCACCATTCCAGAAAATAATCTCCAAGAAAATAATCTCCACCATTTGAGAAAATAATCTCCACCATTTGTTCTGAATCATGTGGTTTGCTAATGTCCTTTTACAGCCACTGTCTCGTGTGATCTCCCGGCCACCTGGAGAGTATCATTATCCCCATTTTACAGGTGAGGACACCGAGGCCTAGAGGGGATAAATGGCACGTGGGAGGCCACCAGCACGTCCGTGGACGAGGTGAGGCCAGAAGAATCCAGGCTCGGATGCCAAACCCCAGGACTGGGTTCTCTCCTTGTCCCTGGCCTGGTCCCTTG
335.3000000000003
total_s

total_seeds:  453
stopped before ungapped extension:  0
stopped before gapped extension:  214
HSPs:  239
BEST SCORING ALIGNMENT: 
327296
327695

query: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACCTGTCCCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCTATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAATGTTGTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGTCGCGATAAGCAATCCTAAAGGACGTGGTGGACACACCAGAGGGCGGGGCACCCAGCCCTGCCGCTGGTGTTCCCGCAGTCCACCCACCCGCGCGCTGTGCCTGTCACTTTCCTCTTTCCTCACCGCTATCGTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA

db: 	
HSP: 	CCCCTCTCCTCCAGGCCTTCCCCAGACCCCATCGGTTAGACCTCAGCACGTGTCTCTGTGGTTACAGGAGTCTGTGGTGGCAGGACGGGCTGGTCAATTTCTAACCGCCACTCACTGACCACACCATGGGAAGGGATCCAGCAACAACGTTTTAGGCCAGGCCTCACAAATGTTCTCAAATAAATCAAACAGAACACCACAGCGGGGCGAGATAAGCAATCCTAAGGAACGTGGTGGACACGCCAGAGGGCGGGGCACCCAGCCCTGCCTCTGGTGTTCCCGCTGTCCACCCGCCCGCGCGCTCTGCCTGTCATTTTCCTCATTCCTCACCGCTATCTTGAGCTGCCTGCAGATCTTGCCTGTTTATTTCTGCTGTGGAAGTCTGGGGTAGGAGAGGGTA
375.1300000000009
total_s

total_seeds:  422
stopped before ungapped extension:  6
stopped before gapped extension:  200
HSPs:  216
BEST SCORING ALIGNMENT: 
64317
64716

query: 	
HSP: 	GACCAATAGTTTCCACTCTCGCCTTCTCACCTCAGGGCCAAGCCTCCAGTAGAGATTCTGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCGCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGAATCAGACCAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGATGGGGGTGTGTGTGGGTTATGGCGAGGGTGTGTGGAGGAGGTTTGACAACAGGTAGCTTGTGGGTAGGACGTTTCTAAAGATTCCTTCCTGTTGAAGTCTTGGCTACTGTGCAATGAGGTTGTGCCTCATTC

db: 	
HSP: 	GACCAATAGGTTCCTCTCTCTCCTTCTCACCTCAGGGCCATGCCTCCAGTAGAGATTCGGAGAAGGCCATTCTTCGAAGGGGAAAGTTCCTCCTGAGCACTCTGGAGGGATTTAGAAATGCCTGCAGGTGAGGGGCCCTCTGTGGTCTGAGGCAGTAGCATGGAGGGTGGGGGTGGAAGGGATTCAGACAAAGGATGAAGTTCTAGAAGCAACACTTGATTTCTGATGCTCTGCTGCATAGTGAGAGGACTTTGGCTGGGGTGGGGGTTGGGGGTGTGTGTGGGTTGTGGAGGGGGTATGTGGAGGAGGTTTGAGGACAGGTAGCTAGGGGGTAGGAAGTTTCTAAAGATTCCCTCCTGGTGAAGTCTTTGCTACTGTGCAATGAGGTTGTGTCTCATTC
373.7799999999997
total_see

total_seeds:  246
stopped before ungapped extension:  5
stopped before gapped extension:  125
HSPs:  116
BEST SCORING ALIGNMENT: 
399313
399712

query: 	
HSP: 	AGCCAAGACTAGAACTGGACAAGAGTTCAGGCTACGTAAGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTTAAAAATGTTTTAAAGGGACATATTTCATCAGGATGTCTGTCACGCCAGCGGTGGAAGCCAACACACCAATTCTCGGAGCCTGTTCCCCCTTCGCGTCAGATCCTCACCCTCAGCCCCTAGGCCCGGAGGCTGGCTGCTTAGCCCAGGGCACAGGAGGGGCCTGCGCCCTGGGCTTGGGGCTGGCATACAGCAGGTGCTCCGTGAACACGGAGGGAACGAGGATCCTGAGCTGATTCGAGAGGATGACAACGGAGAACGTTCGAACAGCTCTCCCGTGTGCTCGGCACCGTGCTGAGTGGATTACCAGCATTAACTCACGT

db: 	
HSP: 	AGCCAAGACTAGAACCTGACAACAGTTCAGGCTACGTATGGCCCCCGTGGGGTTAAGAAAGCTTTCCAGAAATTTCTTTCAAAAATGTTTTAAAGTGACATATTTCATCAGGATGTCTGTCACGCCAGCAGTGGGAGCCAACACACCAATTCTCGGAGGCTGTTCCCCCTGCGCGTCAGATCCACACGCTCAGCCCCCAGGCCCTGAGGCTGGCGGCTTAGCCCAGGGCACGGGAGGGGCCTGCTCCCTGGGCTTGGTGCTGGCACACAGCAGGTGCTCAGTAAACGCTGAGGGAAGGAGGAGGCTGAGCTGATTTGAGAGGATAACAACGGAGAACATTCCTACAGCTCTCCCGTGTGCCGGGCACCGTGCTGAGTGCATTACCTGCATTAACTCACGC
344.37000000000063
[1, 13

In [ ]:
pblast = blast.PBlast()

pblast.set_blast_params(max_decrease_stop=1)

results = test_probabilistic_blast("final_testing_suite",pblast)

pblast.exit()

print(results)